In [23]:
# Create submission script for 2024 by using the receivals from 2024

import pandas as pd
import numpy as np

orders_with_receivals = pd.read_csv("../data_cleaned/orders_with_receivals_detailed.csv", parse_dates=["delivery_date", "created_date_time", "modified_date_time", "date_arrival"])
orders_with_receivals_2024 = orders_with_receivals[orders_with_receivals['date_arrival'].dt.year == 2024]
orders_with_receivals_2024 = orders_with_receivals_2024[['rm_id', 'date_arrival', 'net_weight']]

In [24]:
sample_submission = pd.read_csv("../data/sample_submission.csv")
prediction_mapping = pd.read_csv("../data/prediction_mapping.csv", parse_dates=["forecast_start_date", "forecast_end_date"])
submission = sample_submission.merge(prediction_mapping, on="ID")

In [25]:
print(sample_submission.shape)
print(prediction_mapping.shape)
print(submission.shape)

(30450, 2)
(30450, 4)
(30450, 5)


In [26]:
# For every forecast_start_date and forecast_end_date in submission make it 2024 instead of 2025
submission['forecast_start_date'] = submission['forecast_start_date'].apply(lambda x: x.replace(year=2024))
submission['forecast_end_date'] = submission['forecast_end_date'].apply(lambda x: x.replace(year=2024))

In [22]:

for receival in orders_with_receivals_2024.itertuples():
    rm_id = receival.rm_id
    date_arrival = receival.date_arrival
    net_weight = receival.net_weight
    # Convert date_arrival to naive datetime for comparison
    date_arrival_naive = date_arrival.replace(tzinfo=None)
    submission.loc[
        (submission['rm_id'] == rm_id) & (submission['forecast_end_date'] >= date_arrival_naive),
        'predicted_weight'
    ] += net_weight

In [ ]:
# THIS IS METHOD 2 - MORE CONSERVATIVE - ONLY ADD IF RECEIVAL IS 7 DAYS BEFORE FORECAST END DATE

for receival in orders_with_receivals_2024.itertuples():
    rm_id = receival.rm_id
    date_arrival = receival.date_arrival
    net_weight = receival.net_weight
    date_arrival_naive = date_arrival.replace(tzinfo=None)
    mask = (
        (submission['rm_id'] == rm_id) &
        (submission['forecast_end_date'] >= (date_arrival_naive+pd.Timedelta(days=7)))
    )
    submission.loc[mask, 'predicted_weight'] += net_weight

In [ ]:
# save submission to a csv
submission.to_csv("./submission_2024.csv", index=False)

In [24]:
# Ehhrm some RM_IDS were not there so I wanted to double check .itertuples(), but I just found out that some RM_IDs are seasonal
valid_rm_ids1 = list(orders_with_receivals_2024['rm_id'].unique())
valid_rm_ids2 = list(orders_with_receivals_2024['rm_id'].unique())


count = 0

for receival in orders_with_receivals_2024.itertuples():
    rm_id = receival.rm_id
    date_arrival = receival.date_arrival
    net_weight = receival.net_weight
    # Convert date_arrival to naive datetime for comparison
    if rm_id in valid_rm_ids1:
        valid_rm_ids1.remove(rm_id)
    if rm_id not in valid_rm_ids2:
        count += 1
    

print(count)

1


In [25]:
print(valid_rm_ids1)
print(valid_rm_ids2)

[nan]
[3883.0, 2130.0, 2147.0, 2129.0, 3581.0, 3265.0, 3122.0, 3121.0, 3901.0, 2124.0, 2741.0, 3125.0, 3126.0, 3124.0, 3282.0, 3123.0, 2140.0, 3761.0, 2132.0, 3865.0, 3781.0, 2131.0, 2145.0, 2135.0, 3421.0, 2144.0, 2142.0, 2143.0, 2134.0, 2133.0, nan, 3621.0, 2981.0, 3142.0, 4021.0, 4044.0, 3642.0, 3362.0, 3381.0, 2125.0, 4161.0, 2161.0, 4081.0, 4222.0, 4263.0, 3601.0, 3201.0, 4302.0, 4401.0, 4343.0, 2123.0, 4443.0, 4441.0, 4381.0, 3461.0, 3701.0, 4462.0, 4463.0, 4461.0, 4481.0, 4501.0]


In [5]:
test_submission = pd.read_csv("./submission_2024.csv")
test_submission = test_submission[["ID", "predicted_weight"]]
test_submission["predicted_weight"] = test_submission["predicted_weight"] * 0.8
test_submission.to_csv("./test2024.csv", index=False)